In [4]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')

In [5]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 140kB/s  eta 0:00:01    |███████████████▎                | 103.9MB 4.2MB/s eta 0:00:27
     |████████████████████████████████| 204kB 47.8MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [6]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [7]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [21]:
import random
rdd = sc.parallelize(random.sample(range(1000),100))
print(rdd.take(10))

[368, 369, 626, 824, 460, 994, 632, 179, 154, 486]


In [22]:
# mean
sum = rdd.sum()
n = rdd.count()
mean = sum/float(n)
print(mean)

490.67


In [23]:
# median
sorted = rdd.sortBy(lambda x: x)
sorted_and_index = sorted.zipWithIndex()
sorted_with_index = sorted_and_index.map(lambda x: (x[1],x[0]))
n = sorted_with_index.count()

if n % 2 == 1:
    index = (n-1)/2
    median = sorted_with_index.lookup(index)[0]
else:
    index1 = n/2 - 1
    index2 = n/2
    median = (sorted_with_index.lookup(index1)[0] +  sorted_with_index.lookup(index2)[0]) / 2

print(median)

497.5


In [24]:
# standard deviation
from math import sqrt
sd = sqrt(rdd.map(lambda x: pow(x-mean,2)).sum()/n)
print(sd)

261.41117248503366


In [25]:
# skewness
n = float(n)
skewness = (rdd.map(lambda x: pow(x-mean,3)).sum()/pow(sd,3))/n
skewness

-0.04931414014401959

In [26]:
# kurtosis
kurtosis = (rdd.map(lambda x: pow(x-mean,4)).sum()/pow(sd,4))/n
kurtosis

2.059206389310504

In [27]:
# covariance
rddX = sc.parallelize(random.sample(range(1000),100))
rddY = sc.parallelize(random.sample(range(1000),100))
meanX = rddX.sum()/float(rddX.count())
meanY = rddY.sum()/float(rddY.count())
rddXY = rddX.zip(rddY)
covXY = rddXY.map(lambda x: (x[0] - meanX)*(x[1] - meanY)).sum() / rddXY.count()

# correlation
sdX = sqrt(rddX.map(lambda x: pow(x-meanX,2)).sum()/rddX.count())
sdY = sqrt(rddY.map(lambda x: pow(x-meanY,2)).sum()/rddY.count())
corrXY = covXY/(sdX*sdY)

print(covXY)
print(corrXY)

-4486.401000000001
-0.053319506006586595


In [28]:
# covariance matrix
from pyspark.mllib.stat import Statistics
col1 = sc.parallelize(random.sample(range(1000),100))
col2 = sc.parallelize(random.sample(range(1000),100))
col3 = sc.parallelize(random.sample(range(1000),100))
col4 = sc.parallelize(random.sample(range(1000),100))
data = col1.zip(col2).zip(col3).zip(col4)
data = data.map(lambda x: [x[0][0][0],x[0][0][1],x[0][1],x[1]])
Statistics.corr(data)

array([[ 1.        , -0.17785432, -0.04262465, -0.05200905],
       [-0.17785432,  1.        ,  0.10953086, -0.03486309],
       [-0.04262465,  0.10953086,  1.        ,  0.15173876],
       [-0.05200905, -0.03486309,  0.15173876,  1.        ]])